In [ ]:
!pip install konlpy
!pip install sklearn
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 453 kB 47.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 13.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=f25b418f1f4eac7b4a03162366e5e748df54a5b2a73f57ad34e1fbfb7abce15a
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [1]:
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np
from konlpy.tag import Okt
import re
import pprint
import os

In [2]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

In [3]:
data1=pd.read_csv('핀테크어플리뷰.csv')
data2=pd.read_csv('은행어플리뷰.csv')
#3점을 기준으로 data frame 분리한다. /3점 리뷰는 포함하지 않음  
#data1_1 : 핀테크 어플 리뷰 (부정) -1번째 데이터
#data1_2 : 핀테크 어플 리뷰 (긍정) -2번째 데이터
#data2_1 : 은행 어플 리뷰 (부정)  - 3번째 데이터
#data2_2 : 은행 어플 리뷰 (긍정) - 4번째 데이터

data1_1=data1.loc[data1['별점']<3]
data1_1=data1_1.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
data1_2=data1.loc[data1['별점']>3]
data1_2=data1_2.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

data2_1=data2.loc[data2['별점']<3]
data2_1=data2_1.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
data2_2=data2.loc[data2['별점']>3]
data2_2=data2_2.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')




In [4]:
df=[data1_1,data1_2,data2_1,data2_2]
for i in range(4):
  print(f'{i+1}번째 데이터')
  print(' ')
  df[i]=pd.DataFrame(df[i])
  print(df[i])

1번째 데이터
 
      index    어플명    id      date     연  월   일  별점          이름  \
0         0     토스     1  20220323  2022  3  23   2          HY   
1         5     토스     6  20220505  2022  5   5   1         김지연   
2        10     토스    11  20220518  2022  5  18   1          윤명   
3        15     토스    16  20220317  2022  3  17   1         이창숙   
4        30     토스    31  20220327  2022  3  27   2     WORKOUT   
...     ...    ...   ...       ...   ... ..  ..  ..         ...   
1679   4508  카카오뱅크  2793  20210528  2021  5  28   1         김성민   
1680   4510  카카오뱅크  2795  20210527  2021  5  27   1         황세헌   
1681   4511  카카오뱅크  2796  20210903  2021  9   3   1  Hyunsik Yu   
1682   4513  카카오뱅크  2798  20210312  2021  3  12   1    Master G   
1683   4515  카카오뱅크  2800  20200106  2020  1   6   1         a a   

                                                     내용  
0     한국에서 이용하다가 해외 거주자라 다시 돌아왔는데요. 폰번호 해지되서 이용이 안됐어...  
1     미션이 없어진 곳은 삭제를 제때에 삭제를 해주셔야 하지 않나요? 그리고 미션이 많이...  
2     미션을 바

In [5]:
df[0].head()

,index,어플명,id,date,연,월,일,별점,이름,내용
0,0,토스,1,20220323,2022,3,23,2,HY,한국에서 이용하다가 해외 거주자라 다시 돌아왔는데요. 폰번호 해지되서 이용이 안됐어...
1,5,토스,6,20220505,2022,5,5,1,김지연,미션이 없어진 곳은 삭제를 제때에 삭제를 해주셔야 하지 않나요? 그리고 미션이 많이...
2,10,토스,11,20220518,2022,5,18,1,윤명,미션을 바꾸겠읍니까? 미션을 바꾸겠읍니까? 미션을 바꾸겠읍니까? 도데체 뭘 자꾸 미...
3,15,토스,16,20220317,2022,3,17,1,이창숙,"업데이트했더니, 토스증권 들어갈때 입력했었던 비밀번호 설정이 없어졌어요. 처음 들어..."
4,30,토스,31,20220327,2022,3,27,2,WORKOUT,"현재 느끼는 두가지 불편사항. 첫째, 타행어플에서 내 토스통장으로 입금하면 알림 뜨..."


In [6]:
for i in range(4) :
  df[i]['내용']=df[i]['내용'].apply(lambda x:re.sub(r"\d+", " ", x))
  print(f'{i+1}번째 데이터 내용 column의 첫번째 문서')
  print(df[i]['내용'][0],end='\n')

1번째 데이터 내용 column의 첫번째 문서
한국에서 이용하다가 해외 거주자라 다시 돌아왔는데요. 폰번호 해지되서 이용이 안됐어요. 문의 넣어도 한국폰번호 없으면 이용이 안된다고 딱 짤라 말씀하시길래, 일부러 한국 유심카드 사서 미국으로 우편 받았습니다. 근데 본인인증 문자로 받아서 진행하려는데 본인인증 전화도 해야되네요? 왜 이렇게까지 하는거죠? 해외에 살면 한국 유심칩 있어도 국제전화 수신이 안되거든요. 문자로 인증하면 됐지 진짜 이거때문에 약정걸어서 유심까지 샀는데 짜증나서 미치겠습니다.
2번째 데이터 내용 column의 첫번째 문서
업데이트 후 해외사용, 인증불가/ 안녕하세요. 토스를 몇 년 째 잘 사용해오고 있습니다. 이민 후 지웠다가 이번에 한국에 잠시 들어갔을 때 한국폰번호가 생겨서 다시 깔아서 여태 잘 사용했는데요, 돌아오면서 번호해지를 하였습니다. 업데이트 이후 한국번호가 없어서 인증을 할 수 없습니다. 한국카드는 있지만 받을 수 있는 전화번호가 없구요... 주식이랑 모든 걸 거기서 이용중인데 방법이 있을까요?
3번째 데이터 내용 column의 첫번째 문서
만든 사람! 이 앱을 실제로 사용해봤습니까? 핸드폰 바꾸고 새로 앱깔고 로긴하려니 불가능하네요. 사용하지 말라는 의미인거죠? 인증서 다시 등록하려니 otp 입력하라하고 otp재등록하려니 id/pw로긴으로는 되지도 않고 참고 또 참고 해보려고 했는데.. 이렇게 무식하게 만들어놓은 앱은 증권사앱 다음으로 첨이네요. 은행권이나 카드권은 이 정도는 아닌데...진짜 빡치네요. 별 마이너스로 주는게 없다니 안타깝네요.
4번째 데이터 내용 column의 첫번째 문서
앱은 좋습니다. 다만, 수정이 필요한게 있습니다. 업데이트가 생긴 날 이후에는 공과금 납부시 사용자 주민번호 누락인지 다른것인지 메시지 나오면서 납부안되게 되더군요. 이런 오류는 최신업데이트가 되면 문구가 사라지고 정상적으로 공과금이 납부가 됩니다. 이건 사람들을 오해하게(혼돈하게 만듬) 하기에 수정이 필요해 보입니다.


In [9]:
import konlpy
import re

# tokenization 함수를 만들어둠
def tokenize_korean_text(text):
    text = re.sub(r'[^,.?!\w\s]','', text)  ## ,.?!와 문자+숫자+_(\w)와 공백(\s)만 남김  # 앞에 r을 붙여주면 deprecation warning이 안뜸 (raw string으로 declare)
    
    okt = konlpy.tag.Okt()
    Okt_morphs = okt.pos(text)   # stem=True로 설정하면 동사원형으로 바꿔서 return
    
    words = []
    for word, pos in Okt_morphs:
        #if pos == 'Adjective' or pos == 'Verb' or pos == 'Noun':  # 이 경우에는 형용사, 동사, 명사만 남김
         if pos == 'Noun':
            words.append(word)

    words_str = ' '.join(words)
    return words_str


# review_df['Content']를 하나씩 tokenize해서 list로 저장
tokenized_list1 = []
tokenized_list2 = []
tokenized_list3 = []
tokenized_list4 = []
lists=[tokenized_list1 ,tokenized_list2,tokenized_list3,tokenized_list4]
for i in range(4):
  for text in df[i]['내용']:
    lists[i].append(tokenize_korean_text(text))


  print(f'{i+1}번째 리스트의 토큰의 개수는 {len(lists[i])}이고 첫번째 문서는 \"{lists[i][0]}\"이다.')
print(lists[0])

1번째 리스트의 토큰의 개수는 1684이고 첫번째 문서는 "한국 이용 해외 거주 자라 다시 폰 번호 해지 이용 문의 한국 폰 번호 이용 짤 말씀 일부러 한국 유심 카드 사서 미국 우편 본인 인증 문자 진행 본인 인증 전화 왜 해외 살 한국 유심 칩 국제 전화 수신 문자 인증 진짜 거 때문 약정 유심"이다.
2번째 리스트의 토큰의 개수는 2330이고 첫번째 문서는 "업데이트 후 해외 사용 인증 불가 토스 몇 년 사용 이민 후 이번 한국 잠시 때 한국 폰 번호 다시 여태 사용 번호 업데이트 이후 한국 번호 인증 수 한국 카드 수 전화번호 주식 모든 걸 거기 이용 방법"이다.
3번째 리스트의 토큰의 개수는 2758이고 첫번째 문서는 "사람 이 앱 실제 사용 핸드폰 앱 로 사용 말 의미 인증서 다시 등록 입력 하라 재 등록 로 참고 또 참고 앱 증권사 앱 다음 은행권 카드 이 정도 진짜 별 마이너스"이다.
4번째 리스트의 토큰의 개수는 3820이고 첫번째 문서는 "앱 다만 수정 업데이트 날 이후 공과금 납부 시 사용자 주민 번호 누락 것 메시지 납부 오류 최신 업데이트 문구 정상 공과금 납부 이건 사람 오해 혼돈 만듬 수정"이다.
['한국 이용 해외 거주 자라 다시 폰 번호 해지 이용 문의 한국 폰 번호 이용 짤 말씀 일부러 한국 유심 카드 사서 미국 우편 본인 인증 문자 진행 본인 인증 전화 왜 해외 살 한국 유심 칩 국제 전화 수신 문자 인증 진짜 거 때문 약정 유심', '미션 곳 삭제 제때 삭제 미션 삭제 솤직히 토스 계속 고민 다른 앱 미션 곳 곳 표시 제대로 앱 운영 의문 듭니', '미션 읍 미션 읍 미션 읍 도데 체 뭘 자꾸 미션 읍 글자 토시 하나 미션 읍 장난 질도 정도 껏 몇번 리뷰 개선 하든 개선 생각 의문 해당 미션 완려 외 미션 한글 컴퓨터 답변 달 기전 토시 하나 리뷰 놨는데 일부러 열 답변', '업데이트 토스 증권 때 입력 비밀번호 설정 처음 때 비번 입력 결정 는걸 개별 비번 설정 계좌 기능 비번 설정 그게 무슨 숨 기기 버튼 하나로', '

In [27]:

tokens_1=[]
length=0

for i in range(len(lists[0])):
    tokens_1=lists[0][i].split()
    length+=len(tokens_1)
    tokens_1=[]
    
print(length)
        

30302


In [29]:
tokens_1=lists[0][0].split()
print(tokens_1)

['한국', '이용', '해외', '거주', '자라', '다시', '폰', '번호', '해지', '이용', '문의', '한국', '폰', '번호', '이용', '짤', '말씀', '일부러', '한국', '유심', '카드', '사서', '미국', '우편', '본인', '인증', '문자', '진행', '본인', '인증', '전화', '왜', '해외', '살', '한국', '유심', '칩', '국제', '전화', '수신', '문자', '인증', '진짜', '거', '때문', '약정', '유심']


불용어 제거

In [8]:
stopwords=[]
stopwords = np.append(stopwords,['ㄱ','ㄴ','ㄷ','ㄹ','ㅁ','ㅂ','ㅅ','ㅇ','ㅈ','ㅊ','ㅋ','ㅌ','ㅍ','ㅎ','ㅏ','ㅑ','ㅓ','ㅕ','ㅗ','ㅛ','ㅜ','ㅠ','ㅡ','ㅣ'])
noword1=['토스','카카오뱅크','카카오','카뱅','뱅크','어플','앱','뱅킹','뱅크','카카오 뱅크','갑자기','다른','그냥','최고','아주','매우','계속','제일','바로','항상','가끔','조금','지금','모든','신한',\
         '때문','최악','처음','아예','해도','한번','하나','모바일','이런', '후','이후','걸','다시']
noword2=['은행','국민','국민은행','스타','스타뱅킹','어플','앱','뱅킹','근데','리브','왜','진짜','아니','농협','다만','을','를','그리고','별','진짜','너무','정말','하라','오류','해결','자꾸','왜',]
stopwords = np.append(stopwords,noword1)
stopwords = np.append(stopwords,noword2)
print(stopwords)

['ㄱ' 'ㄴ' 'ㄷ' 'ㄹ' 'ㅁ' 'ㅂ' 'ㅅ' 'ㅇ' 'ㅈ' 'ㅊ' 'ㅋ' 'ㅌ' 'ㅍ' 'ㅎ' 'ㅏ' 'ㅑ' 'ㅓ' 'ㅕ'
 'ㅗ' 'ㅛ' 'ㅜ' 'ㅠ' 'ㅡ' 'ㅣ' '토스' '카카오뱅크' '카카오' '카뱅' '뱅크' '어플' '앱' '뱅킹' '뱅크'
 '카카오 뱅크' '갑자기' '다른' '그냥' '최고' '아주' '매우' '계속' '제일' '바로' '항상' '가끔' '조금'
 '지금' '모든' '신한' '때문' '최악' '처음' '아예' '해도' '한번' '하나' '모바일' '이런' '후' '이후' '걸'
 '다시' '은행' '국민' '국민은행' '스타' '스타뱅킹' '어플' '앱' '뱅킹' '근데' '리브' '왜' '진짜' '아니'
 '농협' '다만' '을' '를' '그리고' '별' '진짜' '너무' '정말' '하라' '오류' '해결' '자꾸' '왜']


In [9]:
for k in range(4):
  for i in range(len(lists[k])):
    for j in range(len(stopwords)):
      lists[k][i]=lists[k][i].replace(stopwords[j],'')
  print(f'불용어를 제거한 {k+1}번째  리스트의 첫번째 문서 : {lists[k][0]}')

불용어를 제거한 1번째  리스트의 첫번째 문서 : 한국 이용 해외 거주 자라  폰 번호 해지 이용 문의 한국 폰 번호 이용 짤 말씀 일부러 한국 유심 카드 사서 미국 우편 본인 인증 문자 진행 본인 인증 전화  해외 살 한국 유심 칩 국제 전화 수신 문자 인증  거  약정 유심
불용어를 제거한 2번째  리스트의 첫번째 문서 : 업데이트  해외 사용 인증 불가  몇 년 사용 이민  이번 한국 잠시 때 한국 폰 번호  여태 사용 번호 업데이트 이 한국 번호 인증 수 한국 카드 수 전화번호 주식   거기 이용 방법
불용어를 제거한 3번째  리스트의 첫번째 문서 : 사람 이  실제 사용 핸드폰  로 사용 말 의미 인증서  등록 입력  재 등록 로 참고 또 참고  증권사  다음 권 카드 이 정도   마이너스
불용어를 제거한 4번째  리스트의 첫번째 문서 :   수정 업데이트 날 이 공과금 납부 시 사용자 주민 번호 누락 것 메시지 납부  최신 업데이트 문구 정상 공과금 납부 이건 사람 오해 혼돈 만듬 수정


In [10]:
for i in range(4):
  print(f'{i+1}번째 데이터의 토큰 개수 {len(lists[i])}')

1번째 데이터의 토큰 개수 1684
2번째 데이터의 토큰 개수 2330
3번째 데이터의 토큰 개수 2758
4번째 데이터의 토큰 개수 3820


In [11]:
d_Corpus=[[],[],[],[]]
corpus=[[],[],[],[]]
for i in range(4):
  for index in range(len(lists[i])):
    corpus[i]=lists[i][index]
    if len(set(corpus[i].split())) <3:  # 같은 단어 1-2개만 반복되는 corpus도 지우기 위해 set()을 사용
      df[i].drop(index,axis='index',inplace=True)
      d_Corpus[i].append(corpus[i])
  for corpus[i] in d_Corpus[i]:
    lists[i].remove(corpus[i])
  
  df[i].reset_index(drop=True,inplace=True)
  print(len(lists[i]))
  print(len(df[i]))


1621
1621
1811
1811
2737
2737
3792
3792


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [13]:
#LDA 는 Count기반의 Vectorizer만 적용 
count_vectorizer = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    # 2개의 문서 미만으로 등장하는 단어는 제외, 전체의 10% 이상으로 자주 등장하는 단어는 제외
    # bigram도 포함
feat_vect=[[],[],[],[]]
for i in range(4):
  feat_vect[i] = count_vectorizer.fit_transform(lists[i])
  print(f'CountVectorizer {i+1}의 Shape: {feat_vect[i].shape}')

CountVectorizer 1의 Shape: (1621, 1000)
CountVectorizer 2의 Shape: (1811, 1000)
CountVectorizer 3의 Shape: (2737, 1000)
CountVectorizer 4의 Shape: (3792, 1000)


In [14]:
#함수와 파라미터 
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=3, n_jobs=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

LatentDirichletAllocation(n_components=3)

In [15]:
 # 토픽 수는 3개로 설정 , 총 4개의 벡터들에 대한 실행 
lda1 = LatentDirichletAllocation(n_components=3)
lda2 = LatentDirichletAllocation(n_components=3)
lda3 = LatentDirichletAllocation(n_components=3)
lda4 = LatentDirichletAllocation(n_components=3)
  
LDAs=[lda1,lda2,lda3,lda4]
for i in range(4):
  LDAs[i].fit(feat_vect[i])

In [16]:
def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vectorizer.get_feature_names()

# Topic별 가장 연관도가 높은 word를 10개만 추출
for i in range(4):
  print(f'{i+1}번째 데이터의 정보')
  display_topics(LDAs[i], feature_names, 10)

1번째 데이터의 정보
Topic # 0
이제 사용 일단 비상 예금 그때 반응 사용 보안 정리 대신 강력
Topic # 1
진행 이벤트 거래 이체 계좌 정보 출시 사용 시간 계좌 조회 자주 기능 배열 굳이 메뉴 메뉴
Topic # 2
생각 이체 등록 인증 문제 사용 전화 지문 패턴 인증서 지문 비대 이벤트 혜택 이체 이용
2번째 데이터의 정보
Topic # 0
어디 화면 전환 이용 이벤트 데이터 메뉴 이용 사용성 부분 확인 사용 업무 사용 사용 메뉴
Topic # 1
최근 인식 어디 확인 사용 전체 보기 몇개 사용 지문 사용 중이 그대로 이벤트 이용
Topic # 2
방식 사용성 이벤트 이용 거나 매번 화면 구성 그룹 파악 공과금 인증서 인증서
3번째 데이터의 정보
Topic # 0
이벤트 메뉴 계좌 송금 부터 계좌 이체 뭔가 이체 잔액 사용 이벤트 로그인 메뉴 변경 불편 사용
Topic # 1
가입 방식 여러가지 이체 잔액 방문 입출금 화이팅 사용 이벤트 안드로이드 경험 방식
Topic # 2
설명 자주 기능 우리 노력 상담 적응 로그인 메뉴 믿음 인증 절차 개인
4번째 데이터의 정보
Topic # 0
등록 인증 지문 확인 패턴 보안 공인 공인 인증서 카드 조회
Topic # 1
개선 속도 업데이트 전체 조회 부분 로딩 접속 간편 정리
Topic # 2
구성 한눈 업무 적금 금융 관리 상품 가입 통합 관련


data1_1의 그래프

In [17]:
import pyLDAvis.sklearn  # 핀테크 부정

pyLDAvis.enable_notebook()
vis1 = pyLDAvis.sklearn.prepare(lda1, feat_vect[0], count_vectorizer)
pyLDAvis.display(vis1)

C:\Users\sangmin\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [18]:
import pyLDAvis.sklearn  # 핀테크긍정

pyLDAvis.enable_notebook()
vis2 = pyLDAvis.sklearn.prepare(lda2, feat_vect[1], count_vectorizer)
pyLDAvis.display(vis2)

In [19]:
import pyLDAvis.sklearn  # 은행 부정

pyLDAvis.enable_notebook()
vis3 = pyLDAvis.sklearn.prepare(lda3, feat_vect[2], count_vectorizer)
pyLDAvis.display(vis3)

In [20]:
import pyLDAvis.sklearn  # 은행 긍정

pyLDAvis.enable_notebook()
vis4 = pyLDAvis.sklearn.prepare(lda4, feat_vect[3], count_vectorizer)
pyLDAvis.display(vis4)

여기까지 일단 완료

In [21]:
# 문서별로, 가장 확률이 높은 topic으로 할당해줌
doc_topics=[[],[],[],[]]
for i in range(4):
  doc_topics[i] = LDAs[i].transform(feat_vect[i])

doc_per_topic_lists = [[],[],[],[]]
topic_most_prs=[[],[],[],[]]
topic_prs=[[],[],[],[]]
doc_topic_df=[[],[],[],[]]
for i in range(4):
  for n in range(doc_topics[i].shape[0]):
    topic_most_prs[i] = doc_topics[i][n].argmax()
    topic_prs[i] = doc_topics[i][n].max()
    doc_per_topic_lists[i].append([n, topic_most_prs[i], topic_prs[i]])
  doc_topic_df[i] = pd.DataFrame(doc_per_topic_lists[i], columns=['Doc_Num', 'Topic', 'Percentage'])
  print(f'{i+1}번째 data의 각 문서별 확률이 가장 높은 토픽과 그 토픽에 해당할 확률')
  print(doc_topic_df[i].head())

1번째 data의 각 문서별 확률이 가장 높은 토픽과 그 토픽에 해당할 확률
   Doc_Num  Topic  Percentage
0        0      2    0.682789
1        1      0    0.761562
2        2      0    0.968502
3        3      2    0.850881
4        4      0    0.669441
2번째 data의 각 문서별 확률이 가장 높은 토픽과 그 토픽에 해당할 확률
   Doc_Num  Topic  Percentage
0        0      1    0.699184
1        1      2    0.786589
2        2      0    0.918239
3        3      1    0.948320
4        4      1    0.526371
3번째 data의 각 문서별 확률이 가장 높은 토픽과 그 토픽에 해당할 확률
   Doc_Num  Topic  Percentage
0        0      1    0.550488
1        1      0    0.708652
2        2      2    0.849426
3        3      0    0.648318
4        4      0    0.978088
4번째 data의 각 문서별 확률이 가장 높은 토픽과 그 토픽에 해당할 확률
   Doc_Num  Topic  Percentage
0        0      2    0.589011
1        1      1    0.527977
2        2      1    0.708808
3        3      1    0.856456
4        4      1    0.586065


In [22]:
for i in range(4):
  doc_topic_df[i] = doc_topic_df[i].join(df[i])
print(doc_topic_df[0].head())


   Doc_Num  Topic  Percentage  index 어플명  id      date     연  월   일  별점  \
0        0      2    0.682789      0  토스   1  20220323  2022  3  23   2   
1        1      0    0.761562      5  토스   6  20220505  2022  5   5   1   
2        2      0    0.968502     10  토스  11  20220518  2022  5  18   1   
3        3      2    0.850881     15  토스  16  20220317  2022  3  17   1   
4        4      0    0.669441     30  토스  31  20220327  2022  3  27   2   

        이름                                                 내용  
0       HY  한국에서 이용하다가 해외 거주자라 다시 돌아왔는데요. 폰번호 해지되서 이용이 안됐어...  
1      김지연  미션이 없어진 곳은 삭제를 제때에 삭제를 해주셔야 하지 않나요? 그리고 미션이 많이...  
2       윤명  미션을 바꾸겠읍니까? 미션을 바꾸겠읍니까? 미션을 바꾸겠읍니까? 도데체 뭘 자꾸 미...  
3      이창숙  업데이트했더니, 토스증권 들어갈때 입력했었던 비밀번호 설정이 없어졌어요. 처음 들어...  
4  WORKOUT  현재 느끼는 두가지 불편사항. 첫째, 타행어플에서 내 토스통장으로 입금하면 알림 뜨...  


In [23]:
#토픽별 문서 수 계산
for i in range(4):
  print(f'{i+1}번째 데이터의 토픽별 문서 수')
  print(doc_topic_df[i].groupby('Topic')[['Doc_Num']].count())

1번째 데이터의 토픽별 문서 수
       Doc_Num
Topic         
0          557
1          473
2          591
2번째 데이터의 토픽별 문서 수
       Doc_Num
Topic         
0          610
1          547
2          654
3번째 데이터의 토픽별 문서 수
       Doc_Num
Topic         
0          978
1          680
2         1079
4번째 데이터의 토픽별 문서 수
       Doc_Num
Topic         
0         1053
1         1181
2         1558


In [24]:
# 토픽별로, 가장 높은 확률로 할당된 문서 top 3 확인
top_pr_topics=[[],[],[],[]]
for i in range(4):
  print(f'{i+1}번째 데이터의 토픽별 top3 문서')
  for topic in range(len(doc_topic_df[i]['Topic'].unique())):
    print(f'Topic # {topic}:  -----------------------------')
    top_pr_topics[i] = doc_topic_df[i][doc_topic_df[i]['Topic'] == topic].sort_values(by='Percentage', ascending=False)
    for j in range(3):
      print(top_pr_topics[i]['내용'].iloc[j],'\n')
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ')



1번째 데이터의 토픽별 top3 문서
Topic # 0:  -----------------------------
장점은 타 증권사에 비해 전혀없고 단점은 넘쳐남. 높은 수수료는 기본 환율은 아침에 딱 한번 적용되고 이후에 환율이 내리던 오르던 아침에 적용된 환율로만 거래해야함. 심각하게 불안정해서 매수 매도 타이밍 잡으면서 그래프보고있는데 갑자기 네트워크문제있다며 주식파트에 아무것도 안뜸. 인터넷은 멀쩡하고 토스내 다른 기능은 정상작동함. 해당버그걸리면 한  분간은 주식차트를 볼수도 거래할수도 없음. 이 버그때문에 매수매도타이밍 놓쳐서 손해본게 한두번이 아님. 증권앱이라기엔 차트나 거래량보기등 모든게 부실함. 거래수량은 실거래에 비해  초정도늦게  초마다 갱신됨. 굳이 해외 장임에도 원화만 크게띄우고 전일종가는 달러로 못보게함. 그외 커뮤기능만들어놓고 관리안하는건 기본. 앱내에 이상한 광고뉴스나 이상한 배너가 크게 표시되며 끌수도 없음. 주식고수의 투자라면서 시드 천이상 월 백이상수익자들의 매도매수를 알려주는데 기준이 이상해서 얼마든 조작가능함. 몇달간평균몇퍼수익자가 훨씬 적절함. 

자동 업데이트 걸어두지 말걸 그랬네요ㅠㅠ 이번 업데이트가 진짜 최악입니다. 가장 문제는 무슨 벽에 턱 막힌 것 같이 답답한 화면 인터페이스.. 처음부터 대문짝만하게 송금이라니.. 한눈에 잘 들어오지도 않고 숫자가 크다고 더 잘보이는게 아닌데ㅠㅠ 회사에서 디자이너 해고한건지? 게다가 계좌한눈에 보기는 더 불편해졌고 원래 개설 순서대로 정렬돼있던게 은근슬쩍 토스에서 가입한 계좌가 먼저 보이게 바뀌었네요.. 기능면에서는 토스머니에 왕창 충전해놓고 송금하는 편리함이 있었는데 이번에 토스머니 충전해도  회 무료밖에 안되도록 바뀌었네요? 전반적으로 완전히 회사입장만 고려한, 수수료에 눈먼 배째라식 업데이트라고 밖에 안느껴집니다. 괜히 적금 걸어놔서 삭제도 못하고.. 충전해둔 토스머니만 다 쓰고 카카오 뱅크로 넘어가려고요. 앱은 적금 끝나자마자 삭제할거에요... 

리뷰가 지워졌네요. 송금관